# TensorFlow2.0教程-保持和读取模型

In [1]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.0.0-alpha0'

导入数据

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

## 1.定义一个模型

In [5]:
def create_model():
    model = keras.Sequential([
        keras.layers.Dense(128, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                 loss=keras.losses.sparse_categorical_crossentropy,
                 metrics=['accuracy'])
    return model
model = create_model()
model.summary()
    

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 128)               100480    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


## 2.checkpoint回调

In [7]:
check_path = '106save/model.ckpt'
check_dir = os.path.dirname(check_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(check_path, 
                                                 save_weights_only=True, verbose=1)
model = create_model()
model.fit(train_images, train_labels, epochs=10,
         validation_data=(test_images, test_labels),
         callbacks=[cp_callback])

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 544/1000 [===============>..............] - ETA: 0s - loss: 2.0658 - accuracy: 0.2831 
Epoch 00001: saving model to 106save/model.ckpt
1000/1000 [==============================] - 1s 855us/sample - loss: 1.8036 - accuracy: 0.4190 - val_loss: 1.3101 - val_accuracy: 0.6700
Epoch 2/10
 800/1000 [=======================>......] - ETA: 0s - loss: 1.0327 - accuracy: 0.7125
Epoch 00002: saving model to 106save/model.ckpt
1000/1000 [==============================] - 0s 132us/sample - loss: 1.0101 - accuracy: 0.7190 - val_loss: 0.8742 - val_accuracy: 0.7650
Epoch 3/10
 768/1000 [======================>.......] - ETA: 0s - loss: 0.7168 - accuracy: 0.7865
Epoch 00003: saving model to 106save/model.ckpt
1000/1000 [==============================] - 0s 113us/sample - loss: 0.7214 - accuracy: 0.7900 - val_loss: 0.7212 - val_accuracy: 0.7950
Epoch 4/10
 992/1000 [============================>.] - ETA: 0s - loss: 0.5918 - accuracy: 0.8367
Epoc

In [9]:
!ls {check_dir}

checkpoint  model.ckpt.data-00000-of-00001  model.ckpt.index


In [10]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 69us/sample - loss: 2.4036 - accuracy: 0.0890
Untrained model, accuracy:  8.90%


In [11]:
model.load_weights(check_path)
loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 47us/sample - loss: 0.4465 - accuracy: 0.8620
Untrained model, accuracy: 86.20%


## 3.设置checkpoint回调

In [12]:
check_path = '106save02/cp-{epoch:04d}.ckpt'
check_dir = os.path.dirname(check_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(check_path,save_weights_only=True, 
                                                 verbose=1, period=5)  # 每5
model = create_model()
model.fit(train_images, train_labels, epochs=10,
         validation_data=(test_images, test_labels),
         callbacks=[cp_callback])

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 1s 1ms/sample - loss: 1.7242 - accuracy: 0.4490 - val_loss: 1.2205 - val_accuracy: 0.6890
Epoch 2/10
1000/1000 [==============================] - 0s 102us/sample - loss: 0.9133 - accuracy: 0.7450 - val_loss: 0.8194 - val_accuracy: 0.7800
Epoch 3/10
1000/1000 [==============================] - 0s 88us/sample - loss: 0.6489 - accuracy: 0.8360 - val_loss: 0.6748 - val_accuracy: 0.8050
Epoch 4/10
1000/1000 [==============================] - 0s 78us/sample - loss: 0.5492 - accuracy: 0.8360 - val_loss: 0.6144 - val_accuracy: 0.8150
Epoch 5/10
  32/1000 [..............................] - ETA: 0s - loss: 0.4468 - accuracy: 0.9062
Epoch 00005: saving model to 106save02/cp-0005.ckpt
1000/1000 [==============================] - 0s 130us/sample - loss: 0.4755 - accuracy: 0.8750 - val_loss: 0.5483 - val_accuracy: 0.8330
Epoch 6/10
1000/1000 [==============================] - 0s 94us/sample - loss

In [14]:
!ls {check_dir}

checkpoint			  cp-0010.ckpt.data-00000-of-00001
cp-0005.ckpt.data-00000-of-00001  cp-0010.ckpt.index
cp-0005.ckpt.index


载入最新版模型

In [16]:
latest = tf.train.latest_checkpoint(check_dir)
print(latest)


106save02/cp-0010.ckpt


In [18]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print('restored model accuracy: {:5.2f}%'.format(acc*100))

1000/1000 [==============================] - 0s 78us/sample - loss: 0.4402 - accuracy: 0.8580
restored model accuracy: 85.80%


## 5.手动保持权重

In [20]:
model.save_weights('106save03/manually_model.ckpt')
model = create_model()
model.load_weights('106save03/manually_model.ckpt')
loss, acc = model.evaluate(test_images, test_labels)
print('restored model accuracy: {:5.2f}%'.format(acc*100))

1000/1000 [==============================] - 0s 69us/sample - loss: 0.4402 - accuracy: 0.8580
restored model accuracy: 85.80%


## 6.保持整个模型

In [22]:
model = create_model()
model.fit(train_images, train_labels, epochs=10,
         validation_data=(test_images, test_labels),
         )
model.save('106save03.h5')

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 0s 240us/sample - loss: 1.7636 - accuracy: 0.4460 - val_loss: 1.2041 - val_accuracy: 0.7230
Epoch 2/10
1000/1000 [==============================] - 0s 82us/sample - loss: 0.9278 - accuracy: 0.7410 - val_loss: 0.7989 - val_accuracy: 0.7880
Epoch 3/10
1000/1000 [==============================] - 0s 97us/sample - loss: 0.6722 - accuracy: 0.7970 - val_loss: 0.6739 - val_accuracy: 0.8110
Epoch 4/10
1000/1000 [==============================] - 0s 110us/sample - loss: 0.5326 - accuracy: 0.8530 - val_loss: 0.6027 - val_accuracy: 0.8170
Epoch 5/10
1000/1000 [==============================] - 0s 88us/sample - loss: 0.4674 - accuracy: 0.8640 - val_loss: 0.5623 - val_accuracy: 0.8270
Epoch 6/10
1000/1000 [==============================] - 0s 91us/sample - loss: 0.3986 - accuracy: 0.8900 - val_loss: 0.5429 - val_accuracy: 0.8370
Epoch 7/10
1000/1000 [==============================] - 0s 87us/samp

In [23]:
new_model = keras.models.load_model('106save03.h5')
new_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 128)               100480    
_________________________________________________________________
dropout_11 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [24]:
loss, acc = model.evaluate(test_images, test_labels)
print('restored model accuracy: {:5.2f}%'.format(acc*100))

1000/1000 [==============================] - 1s 810us/sample - loss: 0.4674 - accuracy: 0.8540
restored model accuracy: 85.40%


## 7.其他导出模型的方法

In [26]:
import time
saved_model_path = "./saved_models/{}".format(int(time.time()))

tf.keras.experimental.export_saved_model(model, saved_model_path)
saved_model_path

W0326 20:00:41.243743 140450529666816 deprecation.py:323] From /home/czy/anaconda3/envs/tf2_0/lib/python3.6/site-packages/tensorflow/python/saved_model/signature_def_utils_impl.py:253: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
W0326 20:00:41.244926 140450529666816 tf_logging.py:161] Export includes no default signature!
W0326 20:00:41.639915 140450529666816 tf_logging.py:161] Export includes no default signature!


'./saved_models/1553601639'

In [27]:
new_model = tf.keras.experimental.load_from_saved_model(saved_model_path)
new_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 128)               100480    
_________________________________________________________________
dropout_11 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [28]:
# 该方法必须先运行compile函数
new_model.compile(optimizer=model.optimizer,  # keep the optimizer that was loaded
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Evaluate the restored model.
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 131us/sample - loss: 0.4674 - accuracy: 0.8540
Restored model, accuracy: 85.40%
